In [17]:
import requests
import pandas as pd
import re 
import csv
import time
from datetime import datetime
import mysql.connector
import pymysql, os, json
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine


In [18]:
now = datetime.today()

In [19]:
#['Central Food Hall', 'Tops market', 'Tops daily', 'Eathai', '', 'Healthiful']

In [20]:
list_province_in_thai = []
list_district_in_thai = []
list_subdistrict_in_thai = []
label = pd.read_excel('C:/Users/Tcct/Desktop/GISDB/code/geocodelabel.xlsx', index_col=None)
for i in range(len(label)):
    list_province_in_thai.append(label['name_th'][i])
    list_district_in_thai.append(label['district_name_th'][i])
    list_subdistrict_in_thai.append(label['subdistrict_name_th'][i])

# clear dup
list_province_in_thai = list(dict.fromkeys(list_province_in_thai))
list_district_in_thai = list(dict.fromkeys(list_district_in_thai))
list_subdistrict_in_thai = list(dict.fromkeys(list_subdistrict_in_thai))


In [21]:
def getSevenBySearchKeywordCentralFoodHall():
    url = "https://corporate.tops.co.th:8080/get-all-store?"
    headers = {'Accept': 'application/json, text/plain, */*'}
    params = {"store_format": "Food Hall" }
    response = requests.get(url, headers=headers , params=params)
    #response = requests.post(url, headers=headers)
    payload = response.json()
    with open(f"../trash/Foodhall{str(now.date())}.json"  , "w") as outfile:
        json.dump(payload, outfile)
        

In [22]:
def getSevenBySearchKeywordTopMarket():
    url = "https://corporate.tops.co.th:8080/get-all-store?"
    headers = {'Accept': 'application/json, text/plain, */*'}
    params = {"store_format": "Tops market" }
    response = requests.get(url, headers=headers , params=params)
    #response = requests.post(url, headers=headers)
    payload = response.json()
    with open(f"../trash/Topsmarket{str(now.date())}.json"  , "w") as outfile:
        json.dump(payload, outfile)

In [23]:
def getSevenBySearchKeywordTopDaily():
    url = "https://corporate.tops.co.th:8080/get-all-store?"
    headers = {'Accept': 'application/json, text/plain, */*'}
    params = {"store_format": "Tops daily" }
    response = requests.get(url, headers=headers , params=params)
    #response = requests.post(url, headers=headers)
    payload = response.json()
    with open(f"../trash/Tops_daily{str(now.date())}.json"  , "w") as outfile:
        json.dump(payload, outfile)


In [24]:
def getSevenBySearchKeywordEathai():
    url = "https://corporate.tops.co.th:8080/get-all-store?"
    headers = {'Accept': 'application/json, text/plain, */*'}
    params = {"store_format": "Eathai" }
    response = requests.get(url, headers=headers , params=params)
    #response = requests.post(url, headers=headers)
    payload = response.json()
    with open(f"../trash/Eathai{str(now.date())}.json"  , "w") as outfile:
        json.dump(payload, outfile)

In [25]:
def getSevenBySearchKeywordHealthiful():
    url = "https://corporate.tops.co.th:8080/get-all-store?"
    headers = {'Accept': 'application/json, text/plain, */*'}
    params = {"store_format": "Healthiful" }
    response = requests.get(url, headers=headers , params=params)
    #response = requests.post(url, headers=headers)
    payload = response.json()
    with open(f"../trash/Healthiful{str(now.date())}.json"  , "w") as outfile:
        json.dump(payload, outfile)

In [26]:
getSevenBySearchKeywordCentralFoodHall()
getSevenBySearchKeywordTopDaily()
getSevenBySearchKeywordTopMarket()
getSevenBySearchKeywordEathai()
getSevenBySearchKeywordHealthiful()

In [27]:
def removeWrongkeyword(word):
    payload_word = word
    if(re.findall(r"\จ+[.]",word)):
        payload_word = re.sub(r"\จ+[.]","จังหวัด",word)
    elif(re.findall(r"\อ+[.]",word)):
        payload_word = re.sub(r"\อ+[.]","อำเภอ",word)
    elif(re.findall(r"\ต+[.]",word)):
        payload_word = re.sub(r"\ต+[.]","ตำบล",word)
    return payload_word
        

In [28]:
def extracteachDetail(word):
    payload_word = word
    if(re.findall(r"\จังหวัด",payload_word)):
        payload_word = re.sub("จังหวัด","",payload_word)
    elif(re.findall(r"\อำเภอ",payload_word)):
        payload_word = re.sub("อำเภอ","",payload_word)
    elif(re.findall(r"\ตำบล",payload_word)):
        payload_word = re.sub("ตำบล","",payload_word)
    elif(re.findall(r"\เขต",payload_word)):
        payload_word = re.sub("เขต","",payload_word)
    elif(re.findall(r"\แขวง",payload_word)):
        payload_word = re.sub("แขวง","",payload_word)
        
    return payload_word

In [29]:
def longdoAddress(lat,lng):
    url = "https://api.longdo.com/map/services/address?"
    params = {'lat': lat , 'lon' : lng , 'key' : '1917e02b4ab264ba547fecdd6e82b4aa'}
    response =requests.get(url, params=params)
    payload = response.json()
    #print("------------------------------------------")
    #print(payload)
    #print("------------------------------------------")
    province = removeWrongkeyword(payload['province'])
    district = removeWrongkeyword(payload['district'])
    subdistrict = removeWrongkeyword(payload['subdistrict'])
    try:
        postcode = payload['postcode']
    except :
        postcode = ""
    
    time.sleep(1)
    
    return province,district,subdistrict,postcode

In [30]:
def getStoretypename(id):
    type_store = {
        "Tops SUPERSTORE": "Hypermarket",
        "Tops market": "Supermarket",
        "Tops daily": "Convenience Stores",
        "Healthiful": "Speciality Store",
        "Eathai": "Speciality Store",
        "Central Food Hall": "Speciality Store"

    }

    return type_store.get(id) 


In [31]:
def getAddressDetail(address,lat,lng):
    
    str_address =  re.sub(r'["]',"",address)
    str_address = str_address.replace("  "," ")
    
    length = len(str_address)
    
    #print("index of: " + str(i))
    '''
    print(lat)
    print(lng)
    print(str_address)
    '''
    #remove newline 
    if("\n" in str_address) :
        str_address = re.sub('\r','',str_address)
        str_address = re.sub('\n',' ',str_address)
    
    temp_fix = str_address.split()
    
    payload_address = ""
    
    if(length == 0):
        #print(length)
        province,district,subdistrict,postcode = longdoAddress(lat,lng)
        temp_fix.append(subdistrict)
        temp_fix.append(district)
        temp_fix.append(province)
        temp_fix.append(postcode)
        #print(temp_fix)
        time.sleep(1)
    
    else :
        
        if(not re.findall("[0-9]", temp_fix[-1])) :

            temp_fix[-1] = removeWrongkeyword(temp_fix[-1])
            temp_fix[-2] = removeWrongkeyword(temp_fix[-2])
            temp_fix[-3] = removeWrongkeyword(temp_fix[-3])
            
            province = extracteachDetail(temp_fix[-1])
            district = extracteachDetail(temp_fix[-2])
            subdistrict = extracteachDetail(temp_fix[-3])
            
        else :
            
            temp_fix[-2] = removeWrongkeyword(temp_fix[-2])
            temp_fix[-3] = removeWrongkeyword(temp_fix[-3])
            temp_fix[-4] = removeWrongkeyword(temp_fix[-4])
            
            
            if(not re.findall(r"\จังหวัด", temp_fix[-2])):
                 temp_fix[-2] = "จังหวัด" +  temp_fix[-2]
                    
            if(temp_fix[-2] == "จังหวัดกรุงเทพมหานคร"):
                if(not re.findall(r"\เขต",temp_fix[-3])):
                    temp_fix[-3] = "เขต" + temp_fix[-3]
            else:
                if(not re.findall(r"\อำเภอ",temp_fix[-3])):
                    temp_fix[-3] = "อำเภอ" + temp_fix[-3]
            
            if(temp_fix[-2] == "จังหวัดกรุงเทพมหานคร"):
                if(not re.findall(r"\ตำบล",temp_fix[-4])):
                    payload_word = "ตำบล" + temp_fix[-4]
            else:
                if(not re.findall(r"\แขวง",temp_fix[-4])):
                    payload_word = "แขวง" + temp_fix[-4]
   
     
            postcode = extracteachDetail(temp_fix[-1])
            province = extracteachDetail(temp_fix[-2])
            district = extracteachDetail(temp_fix[-3])
            subdistrict = extracteachDetail(temp_fix[-4])
            
    for add in temp_fix :
        payload_address = payload_address + add + " "
        
        is_find = False
    
    if(province == "กรุงเทพฯ"):
        province = province.replace("กรุงเทพฯ","กรุงเทพมหานคร")
    if(province not in list_province_in_thai):
        print(str_address)
        is_find = True
        province,district,subdistrict,postcode = longdoAddress(lat,lng)
        postcode = extracteachDetail(postcode)
        province = extracteachDetail(province)
        district = extracteachDetail(district)
        subdistrict = extracteachDetail(subdistrict)
    if(district not in list_district_in_thai):
        if(not is_find):
            print(str_address)
            is_find = True
            province,district,subdistrict,postcode = longdoAddress(lat,lng)
            postcode = extracteachDetail(postcode)
            province = extracteachDetail(province)
            district = extracteachDetail(district)
            subdistrict = extracteachDetail(subdistrict)
    if(subdistrict not in subdistrict):
        if(not is_find):
            print(str_address)
            is_find = True
            province,district,subdistrict,postcode = longdoAddress(lat,lng)
            postcode = extracteachDetail(postcode)
            province = extracteachDetail(province)
            district = extracteachDetail(district)
            subdistrict = extracteachDetail(subdistrict)
    '''
    print("index of: " + str(i))
    print("province: " + province)
    print("district: " + district)
    print("subdistrict: " + subdistrict)
    print("postcode: " + postcode)
    '''
    return payload_address,province,district,subdistrict,postcode
 

In [48]:
pro = open(f'../trash/Foodhall{str(now.date())}.json')
payloadFood = json.load(pro)

length = len(payloadFood)
i = 0

list_payload = []
while i < length :
    name = payloadFood[i]['value']['store_name_th']
    name_en = payloadFood[i]['value']['store_name']
    store_id = payloadFood[i]['value']['internal_store_no']
    address = payloadFood[i]['value']['location']
    address_en = payloadFood[i]['value']['location']
    lat = payloadFood[i]['value']['latitude']
    lng = payloadFood[i]['value']['longitude']
    store_type_name = payloadFood[i]['value']['store_format']
    
    if(lat != "" or lng != ""):
        
        print("th: " + name)
        if(i == 2):
            province = "ภูเก็ต"
            district = "ถลาง"
            subdistrict = "ศรีสุนทร"
            postcode = "83110"
            payload_address = "เลขที่ 8/88 หมู่ที่ 5 ตำบล ศรีสุนทร อำเภอ ถลาง จังหวัดภูเก็ต รหัสไปรษณีย์ 83110"
            
        else:
            payload_address,province,district,subdistrict,postcode = getAddressDetail(address,lat,lng)
        #print("eng: " +  name_en)
        
        store_type = getStoretypename(store_type_name)  
        
        if not re.findall("[ก-ฮ]", name):
            name = name_en
        
        dict_payload = { "name": name,
                        "store_id" : store_id,
                        "name_store" : store_type_name, 
                        "store_type" : store_type ,
                        "address" : payload_address ,
                        "lat" : lat,
                        "lng" : lng,
                        "subdistrict": subdistrict ,
                        "district": district,
                        "province": province,
        }
        
        list_payload.append(dict_payload)
        

        
    print("------------------------------------")
    i = i + 1

if(store_type_name == "Tops Food Hall"):
    with open(f"../trash/Centralfood{str(now.date())}.json"  , "w") as outfile:
        json.dump(list_payload, outfile)

th:  Tops Food Hall Marche Thonglor
------------------------------------
th: Tops Food Hall Sukhumvit 39
------------------------------------
th: Tops Thalang
เลขที่ 8/88 หมู่ที่ 5 ตำบล ศรีสุนทร อำเภอ ถลาง จังหวัดภูเก็ต รหัสไปรษณีย์ 83110


KeyError: 'district'

In [ ]:
pro = open(f'../trash/Tops_daily{str(now.date())}.json')
payloaddaily = json.load(pro)

length = len(payloaddaily)
i = 0

list_payload = []
while i < length :
    name = payloaddaily[i]['value']['store_name_th']
    name_en = payloaddaily[i]['value']['store_name']
    store_id = payloaddaily[i]['value']['internal_store_no']
    address = payloaddaily[i]['value']['location']
    address_en = payloaddaily[i]['value']['location']
    lat = payloaddaily[i]['value']['latitude']
    lng = payloaddaily[i]['value']['longitude']
    store_type_name = payloaddaily[i]['value']['store_format']
    
    if(lat != "" or lng != ""):
        
        print("th: " + name)
        
        if(i == 18 and store_type_name == "Tops daily"):
            payload_address = "888 หมู่5 ถนนพัทยา-นาเกลือ ตําบลนาเกลือ อําเภอบางละมุง จังหวัดชลบุรี 20150"
            province = "ชลบุรี"
            district = "บางละมุง"
            subdistrict = "นาเกลือ"
            postcode = "20150"
        else:
            payload_address,province,district,subdistrict,postcode = getAddressDetail(address,lat,lng)
        #print("eng: " +  name_en)
        
        store_type = getStoretypename(store_type_name)  
        
        if not re.findall("[ก-ฮ]", name):
            name = name_en
        
        dict_payload = { "name": name,
                        "store_id" : store_id,
                        "name_store" : store_type_name, 
                        "store_type" : store_type ,
                        "address" : payload_address ,
                        "lat" : lat,
                        "lng" : lng,
                        "subdistrict": subdistrict ,
                        "district": district,
                        "province": province,
        }
        
        list_payload.append(dict_payload)
        

        
    print("------------------------------------")
    i = i + 1

with open(f"../trash/Topsdaily{str(now.date())}.json"  , "w") as outfile:
    json.dump(list_payload, outfile)

th: ท็อปส์ สาขาหมู่บ้านทรัพย์มั่งคง
------------------------------------
th: ท็อปส์ สาขาซอยแสงใหญ่ บางปลา
------------------------------------
th: ท็อปส์ สาขาสุขาภิบาล 6 บางพลี
------------------------------------
th: ท็อปส์ สาขากิ่่งแก้ว ซอย 12
------------------------------------
th: ท็อปส์ สาขาปรีดีพนมยงค์ซอย 46
------------------------------------
th: ท็อปส์ สาขาซอยนาในร่วมใจ ภูเก็ต
------------------------------------
th: ท็อปส์ สาขาพัทยา จอมเทียน ซอย 10
------------------------------------
th: ท็อปส์ สาขาพัทยา ซอยร้อยหลัง
------------------------------------
th: ท็อปส์ สาขาเสรีอ่อนนุชวิลเลจ
------------------------------------
th: ท็อปส์ สาขาเอแบคบางนา นวมินทร์ ซอย 3
------------------------------------
th: ท็อปส์ สาขามหาวิทยาลัยวลัยลักษณ์
------------------------------------
th: ท็อปส์ เคแอลวิลล่าลาดกระบัง
------------------------------------
th: ท็อปส์ สุขาภิบาล5ซอย74
------------------------------------
th: ท็อปส์ สาขาลาดกระบัง 14/1
------------------------------------
th: ท็อ

In [ ]:
pro = open(f'../trash/Topsmarket{str(now.date())}.json')
payloadmarket = json.load(pro)
length = len(payloadmarket)
i = 0

list_payload = []
while i < length :
    name = payloadmarket[i]['value']['store_name_th']
    name_en = payloadmarket[i]['value']['store_name']
    store_id = payloadmarket[i]['value']['internal_store_no']
    address = payloadmarket[i]['value']['location']
    address_en = payloadmarket[i]['value']['location']
    lat = payloadmarket[i]['value']['latitude']
    lng = payloadmarket[i]['value']['longitude']
    store_type_name = payloadmarket[i]['value']['store_format']
    
    if(lat != "" or lng != ""):
        
        print("th: " + name)
        
    payload_address,province,district,subdistrict,postcode = getAddressDetail(address,lat,lng)
        #print("eng: " +  name_en)
        
    store_type = getStoretypename(store_type_name)  
        
    if not re.findall("[ก-ฮ]", name):
        name = name_en
        
    dict_payload = { "name": name,
                    "store_id" : store_id,
                        "name_store" : store_type_name, 
                        "store_type" : store_type ,
                        "address" : payload_address ,
                        "lat" : lat,
                        "lng" : lng,
                        "subdistrict": subdistrict ,
                        "district": district,
                        "province": province,
        }
    list_payload.append(dict_payload)
        

        
    print("------------------------------------")
    i = i + 1

with open(f"../trash/Tops market{str(now.date())}.json"  , "w") as outfile:
    json.dump(list_payload, outfile)

th: ท็อปส์ สาขาเซ็นทรัล นครราชสีมา
------------------------------------
th: ท็อปส์ สาขาเซ็นทรัลพลาซา ปิ่นเกล้า
------------------------------------
th: ท็อปส์ สาขาเซ็นทรัลพลาซา พระราม 3
------------------------------------
th: ท็อปส์ สาขาเซ็นทรัลพลาซา แกรนด์ พระราม 9
------------------------------------
th: ท็อปส์ สาขาโรบินสัน ฟิวเจอร์ ปาร์ค รังสิต
------------------------------------
th: ท็อปส์ สาขาโรบินสัน สุขุมวิท 19
------------------------------------
th: ท็อปส์ สาขาโรบินสัน บางรัก
------------------------------------
th: ท็อปส์ สาขาซีคอน บางแค
------------------------------------
th: ท็อปส์ สาขาออลซีซันส์
------------------------------------
th: ท็อปส์ สาขาเอสพลานาด
เลขที่ 99 ถนนรัชดาภิเษก แขวงดินแดง เขตดิแดง กรุงเทพมหานคร 10400
------------------------------------
th: ท็อปส์ สาขามาบุญครอง
------------------------------------
th: ท็อปส์ สาขานางลิ้นจี่
22 ถนนนางลิ้นจี่ แขวงช่องนนทรี เขตยานาวา กรุงเทพมหานคร 10120
------------------------------------
th: ท็อปส์ สาขาสุขาภิบาล 3
-----

In [ ]:
pro = open(f'../trash/Eathai{str(now.date())}.json')
payloadeathai = json.load(pro)
length = len(payloadeathai)
i = 0

list_payload = []
while i < length :
    name = payloadeathai[i]['value']['store_name_th']
    name_en = payloadeathai[i]['value']['store_name']
    store_id = payloadeathai[i]['value']['internal_store_no']
    address = payloadeathai[i]['value']['location']
    address_en = payloadeathai[i]['value']['location']
    lat = payloadeathai[i]['value']['latitude']
    lng = payloadeathai[i]['value']['longitude']
    store_type_name = payloadeathai[i]['value']['store_format']
    
    if(lat != "" or lng != ""):
        
        print("th: " + name)
        
    payload_address,province,district,subdistrict,postcode = getAddressDetail(address,lat,lng)
        #print("eng: " +  name_en)
        
    store_type = getStoretypename(store_type_name)  
        
    if not re.findall("[ก-ฮ]", name):
        name = name_en
        
    dict_payload = { "name": name,
                    "store_id" : store_id,
                        "name_store" : store_type_name, 
                        "store_type" : store_type ,
                        "address" : payload_address ,
                        "lat" : lat,
                        "lng" : lng,
                        "subdistrict": subdistrict ,
                        "district": district,
                        "province": province,
        }
    list_payload.append(dict_payload)
        

        
    print("------------------------------------")
    i = i + 1

with open(f"../trash/Eathai{str(now.date())}.json" , "w") as outfile:
    json.dump(list_payload, outfile)

th: อีทไทย เซ็นทรัลเอ็มบาสซี
------------------------------------


In [ ]:
pro = open(f'../trash/Healthiful{str(now.date())}.json')
payloadHealthiful = json.load(pro)
length = len(payloadHealthiful)
i = 0

list_payload = []
while i < length :
    name = payloadHealthiful[i]['value']['store_name_th']
    name_en = payloadHealthiful[i]['value']['store_name']
    store_id = payloadHealthiful[i]['value']['internal_store_no']
    address = payloadHealthiful[i]['value']['location']
    address_en = payloadHealthiful[i]['value']['location']
    lat = payloadHealthiful[i]['value']['latitude']
    lng = payloadHealthiful[i]['value']['longitude']
    store_type_name = payloadHealthiful[i]['value']['store_format']
    
    if(lat != "" or lng != ""):
        
        print("th: " + name)
        
    payload_address,province,district,subdistrict,postcode = getAddressDetail(address,lat,lng)
        #print("eng: " +  name_en)
        
    store_type = getStoretypename(store_type_name)  
        
    if not re.findall("[ก-ฮ]", name):
        name = name_en
        
    dict_payload = { "name": name,
                    "store_id" : store_id,
                        "name_store" : store_type_name, 
                        "store_type" : store_type ,
                        "address" : payload_address ,
                        "lat" : lat,
                        "lng" : lng,
                        "subdistrict": subdistrict ,
                        "district": district,
                        "province": province,
        }
    list_payload.append(dict_payload)
        

        
    print("------------------------------------")
    i = i + 1

with open(f"../trash/healthiful{str(now.date())}.json" , "w") as outfile:
    json.dump(list_payload, outfile)

th: เฮลตี้ฟูล สาขา เดอะคริสตัล
------------------------------------


In [ ]:

'''if(store_type_name == "Tops daily"):
    with open(f"../trash/Tops daily{str(now.date())}.json"   , "w") as outfile:
            json.dump(list_payload, outfile)
            
if(store_type_name == "Tops market"):
    with open(f"../trash/Tops market{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_payload, outfile)
            
if(store_type_name == "Healthiful"):
    with open(f"../trash/Healthiful{str(now.date())}.json" , "w") as outfile:
            json.dump(list_payload, outfile)

if(store_type_name == "Eathai"):
    with open(f"../trash/Eathai{str(now.date())}.json" , "w") as outfile:
            json.dump(list_payload, outfile)

if(store_type_name == "Central Food Hall"):
    with open(f"../trash/Central Food Hall{str(now.date())}.json"   , "w") as outfile:
            json.dump(list_payload, outfile)

if(store_type_name == "Tops SUPERSTORE"):
    with open(f"../trash/Tops SUPERSTORE{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_payload, outfile)'''

'if(store_type_name == "Tops daily"):\n    with open(f"../trash/Tops daily{str(now.date())}.json"   , "w") as outfile:\n            json.dump(list_payload, outfile)\n            \nif(store_type_name == "Tops market"):\n    with open(f"../trash/Tops market{str(now.date())}.json"  , "w") as outfile:\n            json.dump(list_payload, outfile)\n            \nif(store_type_name == "Healthiful"):\n    with open(f"../trash/Healthiful{str(now.date())}.json" , "w") as outfile:\n            json.dump(list_payload, outfile)\n\nif(store_type_name == "Eathai"):\n    with open(f"../trash/Eathai{str(now.date())}.json" , "w") as outfile:\n            json.dump(list_payload, outfile)\n\nif(store_type_name == "Central Food Hall"):\n    with open(f"../trash/Central Food Hall{str(now.date())}.json"   , "w") as outfile:\n            json.dump(list_payload, outfile)\n\nif(store_type_name == "Tops SUPERSTORE"):\n    with open(f"../trash/Tops SUPERSTORE{str(now.date())}.json"  , "w") as outfile:\n         

In [ ]:
df_json =  pd.read_json(f"../trash/Centralfood{str(now.date())}.json")
df_json.to_excel(f"../excel/Central/Centralfoodhall/Centralfoodhall{str(now.date())}.xlsx")
df_json.to_csv(f"../csv/Central/Centralfoodhall/Centralfoodhall{str(now.date())}.csv")

In [ ]:
df_json =  pd.read_json(f"../trash/Topsdaily{str(now.date())}.json")
df_json.to_excel(f"../excel/Central/Topsdaily/Tops daily{str(now.date())}.xlsx")
df_json.to_csv(f"../csv/Central/Topsdaily/Tops daily{str(now.date())}.csv")

In [ ]:
df_json =  pd.read_json(f"../trash/Eathai{str(now.date())}.json")
df_json.to_excel(f"../excel/Central/Eathai/Eathai{str(now.date())}.xlsx")
df_json.to_csv(f"../csv/Central/Eathai/Eathai{str(now.date())}.csv")

In [ ]:
df_json =  pd.read_json(f"../trash/Tops market{str(now.date())}.json")
df_json.to_excel(f"../excel/Central/Topsmarket/Topsmarket{str(now.date())}.xlsx")
df_json.to_csv(f"../csv/Central/Topsmarket/Topsmarket{str(now.date())}.csv")

In [ ]:
df_json =  pd.read_json(f"../trash/healthiful{str(now.date())}.json")
df_json.to_excel(f"../excel/Central/Topsmarket/healthiful{str(now.date())}.xlsx")
df_json.to_csv(f"../csv/Central/Healthiful/healthiful{str(now.date())}.csv")

In [ ]:
#Central
cf = pd.read_csv(f'../csv/Central/Centralfoodhall/Centralfoodhall{str(now.date())}.csv',header=0)
et = pd.read_csv(f'../csv/Central/Eathai/Eathai{str(now.date())}.csv',header=0)
ht = pd.read_csv(f'../csv/Central/Healthiful/Healthiful{str(now.date())}.csv',header=0)
td = pd.read_csv(f'../csv/Central/Topsdaily/Tops daily{str(now.date())}.csv',header=0)
tm = pd.read_csv(f'../csv/Central/Topsmarket/Topsmarket{str(now.date())}.csv',header=0)

engine = create_engine('mysql://root:@localhost:3306/eathai')
with engine.connect() as conn, conn.begin():
    et.to_sql(f'eathai{str(now.date())}', conn, if_exists='append', index=False)
conn.close()

engine = create_engine('mysql://root:@localhost:3306/healthiful')
with engine.connect() as conn, conn.begin():
    ht.to_sql(f'healthiful{str(now.date())}', conn, if_exists='append', index=False)
conn.close()

engine = create_engine('mysql://root:@localhost:3306/topsdaily')
with engine.connect() as conn, conn.begin():
    td.to_sql(f'topsdaily{str(now.date())}', conn, if_exists='append', index=False)
conn.close()

engine = create_engine('mysql://root:@localhost:3306/topsmarket')
with engine.connect() as conn, conn.begin():
    tm.to_sql(f'topsmarket{str(now.date())}', conn, if_exists='append', index=False)
conn.close()

engine = create_engine('mysql://root:@localhost:3306/centralfoodhall')
with engine.connect() as conn, conn.begin():
    cf.to_sql(f'topsfoodhall{str(now.date())}', conn, if_exists='append', index=False)
conn.close()

OperationalError: (MySQLdb.OperationalError) (2002, "Can't connect to server on 'localhost' (10061)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)